In [3]:
import cv2
import numpy as np
#Dleap library to detect the face and then the eyes
import dlib
from math import hypot


#can have different indexes if there are multiple webcams
cap = cv2.VideoCapture(0)

#To detect the face
detector = dlib.get_frontal_face_detector()

#This dat file is basically used to detect points on the face. There are a total of 68 points in our face. The
#eyes ranging from 36. See landmarks_points.png for reference
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#we make this for top to bottom line because we have to draw the line from midpoint of two landmarks. (int)
#is used because pixels cant be divided or float.
def midpoint(p1,p2):
    return (int)((p1.x + p2.x)/2), (int)((p1.y+p2.y)/2)

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, facial_landmarks):
    
        #36 is the start point of eyes and 39 the end point. We draw a line from left to right and from top 
        #to bottom. When top to bottom of eyes length becomes 0, means, we are blinking eyes.
        
        left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
        right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
        center_top = midpoint(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[1]))
        center_bottom = midpoint(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[5]))
        
        
        #for horizontal line
        hor_line = cv2.line(frame, left_point,right_point, (0,255,0), 2 )
        
        #for vertical line
        ver_line = cv2.line(frame, center_top,center_bottom, (0,255,0), 2 )
        
        #hypot is used to calculate length of lines
        hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
        ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
        
        ratio = hor_line_length/ver_line_length
        
        return ratio

while True:
    _,frame = cap.read()
    
    #To obtain grayscale image. Grayscale images are lighter and that is why, we prefer them
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #Array where we have all the faces
    faces = detector(gray)
    
    for face in faces:
        #x,y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        
        landmarks = predictor(gray,face)
        
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41],landmarks)
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47],landmarks)
        
        blinking_ratio = (left_eye_ratio+ right_eye_ratio)/2
        
        if blinking_ratio>5.7:
            cv2.putText(frame,"Blinking",(50,150),font,7,(255,0,0)) 
    
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
        
    cv2.imshow("Frame",frame)
    
cap.release()
cv2.destroyAllWindows()
    

27